## Score, and cross-validated scores 

As we have seen, every estimator exposes a score method that can judge the quality of the fit (or the prediction) on new data. Bigger is better.

In [3]:
from sklearn import datasets, svm
digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target
svm = svm.SVC(C=1, kernel='linear')
(svm.fit(X_digits[:-100], y_digits[:-100])
 .score(X_digits[-100:], y_digits[-100:]))

0.97999999999999998

### KFold cross-validation 

To get a better measure of prediction accuracy (which we can use as a proxy for goodness of fit of the model), we can successively split the data in folds that we use for training and testing: 

In [5]:
import numpy as np

X_folds = np.array_split(X_digits, 3)
y_folds = np.array_split(y_digits, 3)
scores = list()
for k in range(3):
    # We use 'list to copy, in order to 'pop later on
    X_train = list(X_folds)
    
    # Use the kth array as the test set
    X_test = X_train.pop(k)
    # Concatenate the two remaining arrays in the training set
    X_train = np.concatenate(X_train)
    
    # Use the same method for the output training and test labels
    y_train = list(y_folds)
    y_test = y_train.pop(k)
    y_train = np.concatenate(y_train)
    
    # Run the estimator and collect scores
    scores.

In [15]:
np.concatenate?